In [25]:
import pandas as pd
import utils
import plotly.graph_objects as go
import instrument

In [26]:
pair = "USD_CAD"
granularity = "H1"
i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [27]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [28]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
df_ma['MA_100'] = df_ma.mid_c.rolling(window=100).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [29]:
df_ma.sample(10)

,time,mid_o,mid_h,mid_l,mid_c,MA_100
3528,2024-06-25T00:00:00.000000000Z,1.36582,1.36582,1.36540,1.36552,1.369609
3200,2024-06-05T08:00:00.000000000Z,1.36729,1.36814,1.36692,1.36799,1.366107
2382,2024-04-18T04:00:00.000000000Z,1.37509,1.37550,1.37502,1.37536,1.377818
3809,2024-07-10T17:00:00.000000000Z,1.36152,1.36219,1.36116,1.36182,1.362928
558,2024-01-03T05:00:00.000000000Z,1.33274,1.33302,1.33252,1.33274,1.323846
42,2023-11-30T17:00:00.000000000Z,1.35870,1.35878,1.35778,1.35848,1.359993
2386,2024-04-18T08:00:00.000000000Z,1.37580,1.37600,1.37484,1.37544,1.378044
3453,2024-06-19T21:00:00.000000000Z,1.37050,1.37110,1.37046,1.37067,1.373266
1809,2024-03-15T07:00:00.000000000Z,1.35377,1.35459,1.35320,1.35454,1.349119
3598,2024-06-27T22:00:00.000000000Z,1.36984,1.37048,1.36973,1.37048,1.367726


In [30]:
def is_trade(row):
    if row.Deviation >= 0.01 and row.Deviation_PREV < 0.01:
        #This would be a signal to sell at 100 points above the moving average
        return -1
    
    if row.Deviation <= -0.01 and row.Deviation_PREV > -0.01:
        #This would be a signal to buy at 100 points below the moving average
        return 1
    
    #This would be for no trade to take place
    return 0

In [31]:
df_ma['Deviation'] = df_ma.MA_100 - df_ma.mid_c
df_ma['Deviation_PREV'] = df_ma.Deviation.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()

In [32]:
df_trades.head(50)

,time,mid_o,mid_h,mid_l,mid_c,MA_100,Deviation,Deviation_PREV,IS_TRADE
267,2023-12-14T02:00:00.000000000Z,1.34842,1.34850,1.34636,1.34638,1.357631,0.011251,0.009347,-1
273,2023-12-14T08:00:00.000000000Z,1.34754,1.34778,1.34531,1.34648,1.356956,0.010476,0.009526,-1
324,2023-12-18T11:00:00.000000000Z,1.33818,1.33842,1.33624,1.33630,1.347448,0.011148,0.009461,-1
423,2023-12-22T14:00:00.000000000Z,1.32646,1.32670,1.32274,1.32320,1.334067,0.010867,0.007737,-1
454,2023-12-26T21:00:00.000000000Z,1.32002,1.32006,1.31953,1.31956,1.329733,0.010173,0.009869,-1
547,2024-01-02T18:00:00.000000000Z,1.33050,1.33275,1.33050,1.33266,1.322505,-0.010155,-0.008093,1
566,2024-01-03T13:00:00.000000000Z,1.33455,1.33644,1.33452,1.33624,1.324971,-0.011269,-0.009728,1
569,2024-01-03T16:00:00.000000000Z,1.33508,1.33620,1.33432,1.33584,1.325433,-0.010407,-0.009821,1
801,2024-01-17T08:00:00.000000000Z,1.35178,1.35270,1.35122,1.35266,1.342635,-0.010025,-0.009297,1
1120,2024-02-05T15:00:00.000000000Z,1.35212,1.35439,1.35209,1.35379,1.342700,-0.011090,-0.008607,1


In [33]:
#The delta or change between the previous row of data
df_trades["DELTA"] = ( df_trades.mid_c.diff() / i_pair.pipLocation).shift(-1)

#Mutiplying these makes sure that the delta factors in whether we brought or sold with the is_trade
df_trades["GAIN"] = df_trades["DELTA"] * df_trades["IS_TRADE"]

In [34]:
#Total amount of points made from trades
df_trades["GAIN"].sum()

143.00000000000188

In [35]:
df_plot = df_ma.iloc[400:600].copy()

In [36]:
df_plot.shape

(200, 9)

In [37]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
fig.add_trace(go.Scatter(x=df_plot.time, 
    y=df_plot.MA_100,
    line=dict(width=2),
    line_shape='spline',
    name='MA_100'
    ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()